In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("/content/complaints_data.csv")

In [ ]:
qna_data = pd.DataFrame({
    'question': data['complaint_text'],
    'answer': data['response_text'],
    'intent': data['Intent']
})

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the 'intent' column
label_encoder = LabelEncoder()
qna_data['intent_label'] = label_encoder.fit_transform(qna_data['intent'])

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
train_data, test_data = train_test_split(qna_data, test_size=0.2, random_state=42)

# Separate features and labels
X_train = train_data[['question', 'intent_label']]
y_train = train_data['answer']

X_test = test_data[['question', 'intent_label']]
y_test = test_data['answer']

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
from transformers import pipeline
from sklearn.metrics import accuracy_score

# Load pre-trained DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased-distilled-squad')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-cased-distilled-squad')

# Tokenize and encode the training and testing data
X_train_encoded = tokenizer(list(X_train), return_tensors='pt', padding=True, truncation=True)
X_test_encoded = tokenizer(list(X_test), return_tensors='pt', padding=True, truncation=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [ ]:
import torch
question = "I never received my order."
encoded_question = tokenizer(question, return_tensors='pt', padding=True, truncation=True)

# Model prediction
output = model(**encoded_question)
answer_start = torch.argmax(output['start_logits'])
answer_end = torch.argmax(output['end_logits']) + 1
predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded_question['input_ids'][0][answer_start:answer_end]))

# Decode the predicted answer
print("Predicted Answer:", predicted_answer)

Predicted Answer: I never received my order .


In [ ]:
predicted_answer

'I never received my order .'

In [ ]:
question = "I do if my product is damaged?"
encoded_question = tokenizer(question, return_tensors='pt', padding=True, truncation=True)


# Model prediction
output = model(**encoded_question)
answer_start = torch.argmax(output['start_logits'])
answer_end = torch.argmax(output['end_logits']) + 1
predicted_answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(encoded_question['input_ids'][0][answer_start:answer_end])
)

# Decode the predicted answer
print("Predicted Answer:", predicted_answer)

Predicted Answer: do
